# Greeks and Sensitivity Analysis

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

## Load the Libs we need

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as si

# import module
from scipy.stats import norm
from datetime import datetime, timezone, date, time
from dateutil.parser import parse

## Calculating Greeks Using Python

### Black-Scholes Model

In [2]:
class mb_BlackScholes:
    def __init__(self, S, K, r, T, v, q=0, t=0):
        self.S = S  # underlying asset price
        self.K = K  # strike price
        self.r = r  # risk-free interest rate
        self.T = T  # time until expiration in years
        self.v = v  # volatility
        self.q = q  # dividend yield
        self.t = t  # current time
        
        # compute d1 and d2
        self.d1 = ((np.log(self.S / self.K) + (self.r - self.q + 0.5 * self.v ** 2) * (self.T - self.t)) /
                  (self.v * np.sqrt(self.T - self.t)))
        self.d2 = self.d1 - self.v * np.sqrt(self.T - self.t)
        
    def mb_price(self, option_type='call'):
        if option_type == 'call':
            return (self.S * np.exp(-self.q * (self.T - self.t)) * norm.cdf(self.d1) - 
                    self.K * np.exp(-self.r * (self.T - self.t)) * norm.cdf(self.d2))
        elif option_type == 'put':
            return (self.K * np.exp(-self.r * (self.T - self.t)) * norm.cdf(-self.d2) - 
                    self.S * np.exp(-self.q * (self.T - self.t)) * norm.cdf(-self.d1))
    
    def mb_delta(self, option_type='call'):
        if option_type == 'call':
            return np.exp(-self.q * (self.T - self.t)) * norm.cdf(self.d1)
        elif option_type == 'put':
            return -np.exp(-self.q * (self.T - self.t)) * norm.cdf(-self.d1)
        
    def mb_gamma(self):
        return ((np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1)) / 
               (self.S * self.v * np.sqrt(self.T - self.t)))
        
    def mb_theta(self, option_type='call'):
        if option_type == 'call':
            return (-((self.S * np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1) * self.v) / (2 * np.sqrt(self.T - self.t))) - 
                    self.r * self.K * np.exp(-self.r * (self.T - self.t)) * norm.cdf(self.d2) + 
                    self.q * self.S * np.exp(-self.q * (self.T - self.t)) * norm.cdf(self.d1))
        elif option_type == 'put':
            return (-((self.S * np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1) * self.v) / (2 * np.sqrt(self.T - self.t))) + 
                    self.r * self.K * np.exp(-self.r * (self.T - self.t)) * norm.cdf(-self.d2) - 
                    self.q * self.S * np.exp(-self.q * (self.T - self.t)) * norm.cdf(-self.d1))
        
    def mb_vega(self):
        return self.S * np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1) * np.sqrt(self.T - self.t)
    
    def mb_rho(self, option_type='call'):
        if option_type == 'call':
            return self.K * (self.T - self.t) * np.exp(-self.r * (self.T - self.t)) * norm.cdf(self.d2)
        elif option_type == 'put':
            return -self.K * (self.T - self.t) * np.exp(-self.r * (self.T - self.t)) * norm.cdf(-self.d2)
        
    # Second order Greeks
    
    def mb_charm(self, option_type='call'):
        if option_type == 'call':
            return -np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1) * ((2 * (self.r - self.q) * (self.T - self.t) - self.d1 * self.v * np.sqrt(self.T - self.t)) / 
                    (2 * (self.T - self.t) * self.v * np.sqrt(self.T - self.t))) - self.q * np.exp(-self.q * (self.T - self.t)) * norm.cdf(self.d1)
        elif option_type == 'put':
            return -np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1) * ((2 * (self.r - self.q) * (self.T - self.t) - self.d1 * self.v * np.sqrt(self.T - self.t)) / 
                    (2 * (self.T - self.t) * self.v * np.sqrt(self.T - self.t))) + self.q * np.exp(-self.q * (self.T - self.t)) * norm.cdf(-self.d1)
        
    def mb_vomma(self):
        return self.mb_vega() * (self.d1 * self.d2 / self.v)
    
    def mb_vanna(self):
        return -np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1) * self.d2 / self.v
        
    def mb_speed(self):
        return np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1) / (self.S**2 * self.v * np.sqrt(self.T - self.t)) * (self.d1 + self.v * np.sqrt(self.T - self.t))
        
    def mb_zomma(self):
        return self.mb_gamma() * ((self.d1 * self.d2 - 1) / self.v)
    
    def mb_color(self):
        return np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1) / (2 * self.S * self.v * np.sqrt(self.T - self.t)) * (1 + self.d1 * ((2 * (self.r - self.q) * (self.T - self.t) - self.d2 * self.v * np.sqrt(self.T - self.t)) / 
                (2 * self.v * np.sqrt(self.T - self.t))))


### Calculate the Greeks using BlackScholes

In [3]:
# Define option parameters
S = 100  # underlying asset price
K = 105  # strike price
T = 1    # time until expiration in years
r = 0.05 # risk-free interest rate
q = 0.01 # dividend yield
v = 0.20 # volatility

# Calculate the Greeks using the extended BlackScholes class
bs = mb_BlackScholes(S, K, r, T, v, q)

# Print the Greeks for call options
print('Call Option Greeks:')
print('Price: ', bs.mb_price(option_type='call'))
print('Delta: ', bs.mb_delta(option_type='call'))
print('Gamma: ', bs.mb_gamma())
print('Theta: ', bs.mb_theta(option_type='call'))
print('Vega: ', bs.mb_vega())
print('Rho: ', bs.mb_rho(option_type='call'))
print('Charm: ', bs.mb_charm(option_type='call'))
print('Vomma: ', bs.mb_vomma())
print('Vanna: ', bs.mb_vanna())
print('Speed: ', bs.mb_speed())
print('Zomma: ', bs.mb_zomma())
print('Color: ', bs.mb_color())

# Print the Greeks for put options
print('\nPut Option Greeks:')
print('Price: ', bs.mb_price(option_type='put'))
print('Delta: ', bs.mb_delta(option_type='put'))
print('Gamma: ', bs.mb_gamma())
print('Theta: ', bs.mb_theta(option_type='put'))
print('Vega: ', bs.mb_vega())
print('Rho: ', bs.mb_rho(option_type='put'))
print('Charm: ', bs.mb_charm(option_type='put'))
print('Vomma: ', bs.mb_vomma())
print('Vanna: ', bs.mb_vanna())
print('Speed: ', bs.mb_speed())
print('Zomma: ', bs.mb_zomma())
print('Color: ', bs.mb_color())


Call Option Greeks:
Price:  7.491693155007894
Delta:  0.5171512290359106
Gamma:  0.019717640994265942
Theta:  -5.637548457246437
Vega:  39.435281988531884
Rho:  44.223429748583165
Charm:  -0.07299050034183298
Vomma:  -1.5908834261433795
Vanna:  0.28383706062941993
Speed:  5.048685791412178e-05
Zomma:  -0.09938364668440139
Color:  0.010009108342042454

Put Option Greeks:
Price:  8.365799352666059
Delta:  -0.47289860471325745
Gamma:  0.019717640994265942
Theta:  -1.6336438123668557
Vega:  39.435281988531884
Rho:  -55.655659823991805
Charm:  -0.0630900020043413
Vomma:  -1.5908834261433795
Vanna:  0.28383706062941993
Speed:  5.048685791412178e-05
Zomma:  -0.09938364668440139
Color:  0.010009108342042454
